In [1]:
import pandas as pd
import numpy as np

In [3]:
datadir='data/'
name='OnlineRetail.csv'
data=pd.read_csv(datadir+name)
#data.timestamp=data.timestamp.apply(lambda x: x if '.' in x else x+'.00')
data

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,date,time
0,N536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55,N17850,United Kingdom,2010-12-01,08:26:00
1,N536365,71053,WHITE METAL LANTERN,6,3.39,N17850,United Kingdom,2010-12-01,08:26:00
2,N536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75,N17850,United Kingdom,2010-12-01,08:26:00
3,N536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39,N17850,United Kingdom,2010-12-01,08:26:00
4,N536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39,N17850,United Kingdom,2010-12-01,08:26:00
...,...,...,...,...,...,...,...,...,...
541904,N581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,0.85,N12680,France,2011-12-09,12:50:00
541905,N581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2.10,N12680,France,2011-12-09,12:50:00
541906,N581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,4.15,N12680,France,2011-12-09,12:50:00
541907,N581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,4.15,N12680,France,2011-12-09,12:50:00


In [4]:
from src.colinv import profile_dataframe
import json

results = profile_dataframe(data)
types=pd.json_normalize(results)
types

,column_index,column_name,inferred_type,semantic_sense,semantic_confidence,n_unique_if_categorical,numeric_subtype
0,0,InvoiceNo,categorical,other,1.000,25900.0,NaN
1,1,StockCode,categorical,other,0.998,4070.0,NaN
2,2,Description,categorical,other,0.007,4223.0,NaN
3,3,Quantity,numeric,None,0.000,NaN,integer
4,4,UnitPrice,numeric,other,0.935,NaN,float
5,5,CustomerID,categorical,other,1.000,4372.0,NaN
6,6,Country,categorical,other,0.090,38.0,NaN
7,7,date,categorical,phone,1.000,305.0,NaN
8,8,time,categorical,None,0.000,774.0,NaN


In [5]:
for i in ['InvoiceNo', 'CustomerID']:
    types.loc[types[types.column_name == i].index, 'inferred_type']='categorical'	
    types.loc[types[types.column_name == i].index, 'n_unique_if_categorical']=data[i].nunique()
types

,column_index,column_name,inferred_type,semantic_sense,semantic_confidence,n_unique_if_categorical,numeric_subtype
0,0,InvoiceNo,categorical,other,1.000,25900.0,NaN
1,1,StockCode,categorical,other,0.998,4070.0,NaN
2,2,Description,categorical,other,0.007,4223.0,NaN
3,3,Quantity,numeric,None,0.000,NaN,integer
4,4,UnitPrice,numeric,other,0.935,NaN,float
5,5,CustomerID,categorical,other,1.000,4372.0,NaN
6,6,Country,categorical,other,0.090,38.0,NaN
7,7,date,categorical,phone,1.000,305.0,NaN
8,8,time,categorical,None,0.000,774.0,NaN


In [6]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice',
       'CustomerID', 'Country', 'date', 'time'],
      dtype='object')

In [7]:
numcol=types[types.inferred_type=='numeric'].column_name.values
for c in numcol:
    l=len(data[data[c].isna()])
    if l>0:
        data[c]=data[c].fillna(value='-1.')
    print(c, l)

Quantity 0
UnitPrice 0


In [8]:
data['date']=pd.to_datetime(data['date'])

In [9]:
dt=dict(types.inferred_type.value_counts())
dt

{'categorical': 7, 'numeric': 2}

In [24]:
onehot_cols = types[(types.inferred_type=='categorical')&(types.n_unique_if_categorical<=111)].column_name.values
onehot_cols = np.concatenate([onehot_cols, types[(types.inferred_type=='boolean')].column_name.values])
cat_feat_names = types[(types.inferred_type=='categorical')&(types.n_unique_if_categorical>=15)].column_name.values
num_feat_names = types[types.inferred_type=='numeric'].column_name.values
log1p_transform_cols = []
date_feature = 'date'
time_feature = ''
client_id = 'CustomerID'
mcc_name = 'Country'

latent_dim = {'onehot': len(onehot_cols), 'categorical': len(cat_feat_names), 'numerical': len(num_feat_names)}
latent_dim['cv']=sum(latent_dim.values())
latent_dim 

{'onehot': 1, 'categorical': 7, 'numerical': 2, 'cv': 10}

In [25]:
from src.embedder import embeddings
X_emb, X_oh, cond_vector, synth_date, scaler_cat, scaler_onehot, scaler_num, cv_params, scaler, round_array = \
            embeddings(data, cat_feat_names, num_feat_names, onehot_cols, date_feature, time_feature, client_id, latent_dim, device='cpu', load=False, epochs=20)

Loss E_oh: 1.2478831195039675e-05: 100%|████████| 20/20 [13:09<00:00, 39.48s/it]
/home/algomath/.local/lib/python3.10/site-packages/rdt/transformers/categorical.py:132: FutureWarning: The behavior of value_counts with object-dtype is deprecated. In a future version, this will *not* perform dtype inference on the resulting index. To retain the old behavior, use `result.index = result.index.infer_objects()`
  freq = data.value_counts(normalize=True, dropna=False)
Loss E_num: 1.5692684485202335e-08: 100%|███████| 20/20 [10:37<00:00, 31.87s/it]


Optimizing poisson intensity...


Loss E_cv: 0.000019050: 100%|███████████████████| 20/20 [10:19<00:00, 31.00s/it]


In [26]:
from src.embedder import train
dim_noise = 20
generator, supervisor, loss_array = train(X_emb, cond_vector, latent_dim, dim_noise, epochs=40, load=False)

Discriminator Loss: -0.00291 || Discriminator 2 Loss: -0.00023 || Generator Loss


In [27]:
from src.generator import sample, inverse_transform
n_samples = len(X_emb[:])
synth_data, synth_date, params = sample(n_samples, generator, supervisor, dim_noise, cond_vector, X_emb, cv_params['encoder'], data,\
                                date_feature, client_id, cv_params=cv_params, device='cpu')

In [35]:
X_emb1 = scaler.inverse_transform(X_emb)
synth_data = scaler.inverse_transform(synth_data)

In [36]:
synth_data

array([[-0.9643227 ,  0.979472  , -0.91331834, ..., -0.8376441 ,
         0.40484118,  0.17527482],
       [-0.9670281 ,  0.97945774, -0.9132644 , ..., -0.8376638 ,
         0.40513575,  0.17559166],
       [-0.9833614 ,  0.9791672 , -0.9128053 , ..., -0.83699805,
         0.40563327,  0.179973  ],
       ...,
       [ 0.52708083,  0.9785789 ,  0.44394085, ...,  0.50939953,
         0.41847196,  0.12372237],
       [ 0.6538528 ,  0.9772181 ,  0.44852096, ...,  0.51160824,
         0.3603822 ,  0.12368198],
       [ 0.7196483 ,  0.9757385 ,  0.4491711 , ...,  0.51173365,
         0.3361225 ,  0.13369983]], dtype=float32)

In [37]:
synth_data.shape

(541909, 10)

In [38]:
import sys
num_feat_names
data[num_feat_names].apply(lambda x: round(x, sys.float_info.dig - 1))

,Quantity,UnitPrice
0,6,2.55
1,6,3.39
2,8,2.75
3,6,3.39
4,6,3.39
...,...,...
541904,12,0.85
541905,6,2.10
541906,4,4.15
541907,4,4.15


In [39]:
synth_df = inverse_transform(synth_data, latent_dim, X_oh.columns, scaler_onehot, scaler_cat, scaler_num, cat_feat_names,
                             mcc_name, num_feat_names, True, synth_date, time_feature, round_array)

In [33]:
synth_df

,Country,InvoiceNo,StockCode,Description,CustomerID,Country,date,time,Quantity,UnitPrice,date
0,United Kingdom,N559923,22847,SET/3 ROSE CANDLE IN JEWELLED BOX,N13969,United Kingdom,1310515200000000000,15:16:00,-291,-124.428,2011-03-02
1,United Kingdom,N571500,85039B,NUMBER TILE VINTAGE FONT 4,N14507,United Kingdom,1318809600000000000,15:59:00,-179,-76.554,2011-06-23
2,United Kingdom,N566928,22385,FLOWER VINE RAFFIA FOOD COVER,N14572,United Kingdom,1316044800000000000,12:39:00,-224,-95.679,2011-04-12
3,United Kingdom,N546667,84378,72 SWEETHEART FAIRY CAKE CASES,N15287,United Kingdom,1300147200000000000,17:22:00,-424,-157.524,2011-01-28
4,United Kingdom,N570776,22817,GARLAND WITH HEARTS AND BELLS,N15416,United Kingdom,1318377600000000000,13:30:00,-190,-81.068,2011-06-10
...,...,...,...,...,...,...,...,...,...,...,...
541904,United Kingdom,NC566011,21360,SET3 BOOK BOX GREEN GINGHAM FLOWER,N14040,United Kingdom,1315440000000000000,10:34:00,-236,-100.992,2011-04-03
541905,United Kingdom,N579297,23196,SET OF 12 MINI LOAF BAKING CASES,N16049,Germany,1322524800000000000,11:00:00,7,1.925,2011-11-28
541906,United Kingdom,N560635,85132A,RED SPOTTY BISCUIT TIN,N14577,United Kingdom,1311120000000000000,15:34:00,-270,-115.540,2011-03-02
541907,United Kingdom,N538167,22544,POSTAGE,N15287,United Kingdom,1291852800000000000,11:29:00,-767,-225.566,2010-12-05


In [34]:
synth_df.columns

Index(['Country', 'InvoiceNo', 'StockCode', 'Description', 'CustomerID',
       'Country', 'date', 'time', 'Quantity', 'UnitPrice', 'date'],
      dtype='object')